# Extraindo Dados do Twitter com Python


Uma aplicação para monitorar os principais resultados do Twitter e seu devido volume na rede social ao longo do tempo. O objetivo é saber as principais tendências e o que estão falando sobre o assunto.

+ [Search Tweets](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets#)
+ [Buscando tweets com Python](https://www.alura.com.br/artigos/buscando-tweets-com-python)

In [1]:
!pip install geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 4.5 MB/s 


## Importando Bibliotecas

In [2]:
import geocoder
import toml
import tweepy
import json
import pandas as pd

## Arquivo com as Chaves de Acesso ao Twitter

In [5]:
# abrir o arquivo com as credenciais de acesso
with open('config.toml') as config:
  # ler o arquivo e salvar as chaves nas variáveis
  config = toml.loads(config.read())
  APP_NAME = config['APP_NAME']
  API_KEY = config['API_KEY']
  API_KEY_SECRET = config['API_KEY_SECRET']
  ACCESS_TOKEN = config['ACCESS_TOKEN']
  ACCESS_TOKEN_SECRET = config['ACCESS_TOKEN_SECRET']

KeyError: ignored

In [ ]:
APP_NAME

## Autenticando no Twitter

In [ ]:
# função para autenticação do usuário
auth = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
# função para acesso ao app com os tokens
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
# autenticação na API do Twitter
api = tweepy.API(auth)

## Top Trends Twitter

In [ ]:
# Verificando as localizações disponíveis para o trends
available_loc = api.trends_available()
df = pd.json_normalize(available_loc)
df.head(3)

In [ ]:
# Filtrando Brasil nas localizações disponíveis
df.query('country == "Brazil"')

In [ ]:
# Definindo a localização
loc = geocoder.osm('Brazil')
print(loc)

In [ ]:
# Buscando o id da localização basedado na lat e long
closest_loc = api.trends_closest(loc.lat,loc.lng)
print(json.dumps(closest_loc,indent=1))

In [ ]:
# Montando dataset do Top trends da localização informada
trends  = api.trends_place(closest_loc[0]["woeid"])

In [ ]:
# Verificando os dados das Top trends selecionadas
print('Data:',trends[0]['as_of'])
print('Data Trend:',trends[0]['created_at'])
print('Location:',trends[0]['locations'][0]['name'])

In [ ]:
# Exibindo os Top Trends Brasil
df_trends = pd.json_normalize(trends[0]['trends'])
df_trends.head()

## Extraindo Dados do Twitter

In [ ]:
# Realizando uma busca de dados pelo Top Trend
results = api.search(q='Cuca')

# Organizando os dados
users = []
tweets = []
for tweet in results:
  users.append(tweet.user.name)
  tweets.append(tweet.text)
  print(f'User: {tweet.user.name} | Tweet: {tweet.text}')

In [ ]:
# Montando um dataset com os tweets extraidos do Twitter
df_tweets = pd.DataFrame()
df_tweets['users'] = users
df_tweets['tweets'] = tweets
df_tweets.head()